# import modules

In [1]:
import librosa
import os
import pandas as pd
import librosa
import glob
import librosa.display
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

# Define targeting words

In [13]:
words = ['one','two','three','four','five','six','seven','eight','nine','zero']

# Generate DataFrame of filepath -> label

In [5]:
data = []

for word in words:
    for file in os.listdir('./words/sc/'+word):
        data.append([word+'/'+file,word])

labels = pd.DataFrame(data,columns=['filepath','label'])

In [6]:
labels

,filepath,label
0,one/988e2f9a_nohash_0.wav,one
1,one/8d4cdc60_nohash_1.wav,one
2,one/93ec8b84_nohash_0.wav,one
3,one/6272b231_nohash_1.wav,one
4,one/439c84f4_nohash_1.wav,one
...,...,...
23661,zero/a60a09cf_nohash_1.wav,zero
23662,zero/2c7c33e8_nohash_0.wav,zero
23663,zero/540d8427_nohash_0.wav,zero
23664,zero/173ae793_nohash_1.wav,zero


# Parse feautres

In [55]:
def parser(row):
   filepath = row.filepath
   label = row.label
   # function to load files and extract features
   file_name = os.path.join(os.path.abspath('./words/sc'), filepath)
   # print(filename)
   # handle exception to check if there isn't a file which is corrupted
   try:
      # here kaiser_fast is a technique used for faster extraction
      X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
      # we extract mfcc feature from data
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=60).T,axis=0) 
   except Exception as e:
      print("Error encountered while parsing file: ", file)
      return [None,None]
 
   feature = mfccs
 
   return [feature, label]

# THIS CELL RUNS LOOOOOONG

In [56]:
temp = labels.apply(parser, axis=1)

# Prepare Training set

In [58]:
X.shape

(23666, 60)

In [59]:
y.shape

(23666, 10)

In [19]:
df = pd.read_csv('./words/number_40_input.csv')

In [20]:
df['feature']=df['feature'].map(lambda s: list(map(float,s.strip(' []').split())))

In [21]:
# df=pd.DataFrame(temp.to_list(),columns = ['feature','label'])
X = np.array(df.feature.tolist())
y = np.array(df.label.tolist())
lb = LabelEncoder()
y = np_utils.to_categorical(lb.fit_transform(y))

In [22]:
y.shape

(23666, 10)

In [39]:
def make_Model(batch_size):
    filter_size = 2
    num_labels = len(words)
    # build model
    model = Sequential()

    model.add(Dense(128, input_shape=(40,)))
    model.add(Activation('relu'))
#     model.add(Dropout(0.5))

    model.add(Dense(128))
    model.add(Activation('relu'))
#     model.add(Dropout(0.5))
    
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

def train_Model(split,batch_size=32):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=22)
    model = make_Model(batch_size)
    model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test))
    return model

In [15]:
model1 = train_Model(0.2)

Epoch 1/20
592/592 [==============================] - 1s 1ms/step - loss: 6.6432 - accuracy: 0.1065 - val_loss: 2.3027 - val_accuracy: 0.1003
Epoch 2/20
592/592 [==============================] - 1s 1ms/step - loss: 2.3207 - accuracy: 0.1015 - val_loss: 2.3027 - val_accuracy: 0.0965
Epoch 3/20
592/592 [==============================] - 1s 994us/step - loss: 2.3101 - accuracy: 0.0990 - val_loss: 2.3027 - val_accuracy: 0.0965
Epoch 4/20
592/592 [==============================] - 1s 996us/step - loss: 2.3051 - accuracy: 0.1062 - val_loss: 2.2697 - val_accuracy: 0.1409
Epoch 5/20
592/592 [==============================] - 1s 985us/step - loss: 2.2723 - accuracy: 0.1282 - val_loss: 2.2422 - val_accuracy: 0.1580
Epoch 6/20
592/592 [==============================] - 1s 999us/step - loss: 2.2465 - accuracy: 0.1420 - val_loss: 2.2133 - val_accuracy: 0.1684
Epoch 7/20
592/592 [==============================] - 1s 1ms/step - loss: 2.2223 - accuracy: 0.1578 - val_loss: 2.1243 - val_accuracy: 0.203

In [16]:
train_Model(0.5)

Epoch 1/20
370/370 [==============================] - 1s 2ms/step - loss: 8.9307 - accuracy: 0.1088 - val_loss: 2.3028 - val_accuracy: 0.0972
Epoch 2/20
370/370 [==============================] - 0s 1ms/step - loss: 2.3470 - accuracy: 0.1074 - val_loss: 2.3026 - val_accuracy: 0.0981
Epoch 3/20
370/370 [==============================] - 0s 1ms/step - loss: 2.3138 - accuracy: 0.1141 - val_loss: 2.2882 - val_accuracy: 0.1146
Epoch 4/20
370/370 [==============================] - 1s 1ms/step - loss: 2.2803 - accuracy: 0.1269 - val_loss: 2.2418 - val_accuracy: 0.1601
Epoch 5/20
370/370 [==============================] - 0s 1ms/step - loss: 2.2584 - accuracy: 0.1440 - val_loss: 2.2054 - val_accuracy: 0.1677
Epoch 6/20
370/370 [==============================] - 0s 1ms/step - loss: 2.2372 - accuracy: 0.1508 - val_loss: 2.1778 - val_accuracy: 0.1832
Epoch 7/20
370/370 [==============================] - 0s 1ms/step - loss: 2.2219 - accuracy: 0.1601 - val_loss: 2.1476 - val_accuracy: 0.1975
Epoch 

In [40]:
history = train_Model(0.2,32)

Epoch 1/20
592/592 [==============================] - 1s 905us/step - loss: 3.1822 - accuracy: 0.3031 - val_loss: 1.7769 - val_accuracy: 0.3982
Epoch 2/20
592/592 [==============================] - 0s 761us/step - loss: 2.0105 - accuracy: 0.3777 - val_loss: 2.5982 - val_accuracy: 0.3088
Epoch 3/20
592/592 [==============================] - 0s 761us/step - loss: 1.9674 - accuracy: 0.3899 - val_loss: 1.7664 - val_accuracy: 0.4284
Epoch 4/20
592/592 [==============================] - 0s 744us/step - loss: 1.7548 - accuracy: 0.4236 - val_loss: 1.8422 - val_accuracy: 0.4301
Epoch 5/20
592/592 [==============================] - 0s 749us/step - loss: 1.7061 - accuracy: 0.4264 - val_loss: 1.6108 - val_accuracy: 0.4542
Epoch 6/20
592/592 [==============================] - 0s 761us/step - loss: 1.6180 - accuracy: 0.4494 - val_loss: 1.5570 - val_accuracy: 0.4597
Epoch 7/20
592/592 [==============================] - 0s 746us/step - loss: 1.5734 - accuracy: 0.4598 - val_loss: 1.5811 - val_accuracy:

In [54]:
df.to_csv('./words/number_40_input.csv', index=False)